In [1]:
import pandas as pd
import geopandas as gpd

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf_br = gpd.read_file('data/censo_2010/BR_Setores_2010.gpkg')

In [3]:
_ = gdf_br.to_crs(epsg='5880', inplace=True)

In [4]:
df = pd.read_csv('resultados/modelo_preditivo.csv.zip')

/tmp/ipykernel_11827/1623530861.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resultados/modelo_preditivo.csv.zip')


In [5]:
df.shape, gdf_br.shape

((310120, 37), (316574, 16))

In [6]:
gdf_br.loc[:, 'Cod_setor'] = gdf_br.CD_GEOCODI.astype('int')

In [7]:
gdf_br.Cod_setor

0         110009812000003
1         110009815000001
2         110009815000002
3         110009815000003
4         110009815000004
               ...       
316569    280760005000030
316570    280760005000031
316571    280760005000032
316572    280760005000033
316573    280760005000028
Name: Cod_setor, Length: 316574, dtype: int64

In [8]:
df.Cod_setor

0         120001305000001
1         120001305000002
2         120001305000003
3         120001305000004
4         120001305000005
               ...       
310115    170210905000098
310116    170210905000099
310117    170210905000100
310118    170210905000101
310119    170210905000102
Name: Cod_setor, Length: 310120, dtype: int64

In [9]:
gdf_br_joined = gdf_br.set_index('Cod_setor').join(df.set_index('Cod_setor'), how='left', lsuffix='_df')

In [10]:
gdf_br_joined.loc[:, 'legenda'] = 'Setores Comuns'
gdf_br_joined.loc[gdf_br_joined.subnormal_LDA.isna(), 'legenda'] = 'Sem Informação'

In [11]:
subnormal = (gdf_br_joined.tipo_do_setor == 1)

In [12]:
gdf_br_joined.loc[subnormal, 'legenda'] = 'Subnormais'

In [13]:
precarios = (gdf_br_joined.subnormal_LDA == 1.0)

In [14]:
gdf_br_joined.loc[(precarios & ~subnormal), 'legenda'] = 'Precários'

In [15]:
rurais = (gdf_br_joined.TIPO == 'RURAL')

In [16]:
gdf_br_joined.loc[(rurais & ~precarios & ~subnormal), 'legenda'] = 'Rurais'

In [17]:
# gdf_br_joined.subnormal_LDA.isna().value_counts()

In [18]:
gdf_br_joined.legenda.value_counts()

Setores Comuns    215103
Rurais             75626
Subnormais         15815
Sem Informação      6657
Precários           3373
Name: legenda, dtype: int64

In [19]:
gdf_br_joined.to_file('resultados/assentamentos_precarios_BR.gpkg', driver='GPKG')

In [20]:
cores = {
    "Setores Comuns": "#E9E2E7",
    "Rurais": "#C0D265",
    "Precários": "#F8BD09",
    "Subnormais": "#EA1C23",
    "Sem Informação": "#7E7C82"
}

In [21]:
# Apresentação do método

## Introdução
    # Objetivo rodar o modelo criado para 2010 a partir do modelo proposto pelo CEM
    # Aguardar os dados de 2022
## Método
    # Dimensões
        # as 3 dimensões que ficaram de fora
    # Modelos: LDA, RandomForest, SVM
    # agregados espacialmente: RM, UF e o restante MacroRegião
## Resultados
    # Modelo roda para todo BR
    # Performance de cada um dos modelos
    # percentual
    # Alguns exemplos
    # Resultado visual município de SP comparando com favelas, corticos, etc
## Próximos passos
    # Publicação de um Dash
        # por Município e RM
        # publicação de mapas e tabelas

